# Vacation Py

In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

#  Store Part I results into DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
cities_df = pd.read_csv('../WeatherPy/WeatherPy.csv')
# cities_df.dropna(inplace == True)
cities_df

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Zarichne,51.8126,26.1290,22.96,98,89,7.87,UA,1611943092
1,1,Mataura,-46.1927,168.8643,39.43,95,32,1.90,NZ,1611943092
2,2,Ushuaia,-54.8000,-68.3000,39.20,65,75,25.32,AR,1611942995
3,3,New Norfolk,-42.7826,147.0587,61.00,83,76,1.92,AU,1611942498
4,4,Busselton,-33.6500,115.3333,64.00,80,0,3.00,AU,1611943093
...,...,...,...,...,...,...,...,...,...,...
556,556,Ust-Nera,64.5667,143.2000,-16.24,79,100,2.71,RU,1611943173
557,557,Rumoi,43.9344,141.6428,26.42,94,100,8.93,JP,1611943173
558,558,Upata,8.0086,-62.3989,88.00,60,62,3.00,VE,1611943173
559,559,Sumbe,-11.2061,13.8437,78.39,77,9,9.57,AO,1611943173


#  Humidity Heatmap

Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitidue and longitude in locations and humidity in weights
locations = cities_df[["Latitude", "Longitude"]]
weights = cities_df["Humidity"]

#  Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights,dissipating=False, max_intensity=10, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

# Export to png
plt.savefig("Heatmap_Cities")

<Figure size 432x288 with 0 Axes>

#  Hotel Map

Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [6]:
# Narrow down the DataFrame to find your ideal weather condition
ideal_weather_df = cities_df

# A max temperature lower than 80 degrees but higher than 70
ideal_weather_df = ideal_weather_df.loc[(ideal_weather_df["Max Temp"] < 80) & (ideal_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Wind Speed"] < 10]

# Zero cloudiness
ideal_weather_df = ideal_weather_df.loc[ideal_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

ideal_weather_df

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,Najrān,17.4924,44.1277,75.20,15,0,4.61,SA,1611943100
179,179,Pisco,-13.7000,-76.2167,77.00,69,0,9.22,PE,1611943114
238,238,Adrar,20.5022,-10.0711,77.34,16,0,7.70,MR,1611943123
254,254,Tessalit,20.1986,1.0114,72.77,17,0,9.46,ML,1611943125
427,427,Chato,-2.6378,31.7669,70.25,70,0,0.67,TZ,1611943152
439,439,Abu Dhabi,24.4667,54.3667,71.60,64,0,4.61,AE,1611943097
452,452,Araouane,18.9048,-3.5265,77.45,16,0,7.96,ML,1611943156
505,505,Le Port,-20.9373,55.2919,78.80,73,0,9.22,RE,1611943164


In [7]:
# Store ideal weather Data Frame into variable called hotel_df
hotel_df = ideal_weather_df

# Add hotel name column
hotel_df["Hotel Name"] = ""
hotel_df

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
58,58,Najrān,17.4924,44.1277,75.20,15,0,4.61,SA,1611943100,
179,179,Pisco,-13.7000,-76.2167,77.00,69,0,9.22,PE,1611943114,
238,238,Adrar,20.5022,-10.0711,77.34,16,0,7.70,MR,1611943123,
254,254,Tessalit,20.1986,1.0114,72.77,17,0,9.46,ML,1611943125,
427,427,Chato,-2.6378,31.7669,70.25,70,0,0.67,TZ,1611943152,
439,439,Abu Dhabi,24.4667,54.3667,71.60,64,0,4.61,AE,1611943097,
452,452,Araouane,18.9048,-3.5265,77.45,16,0,7.96,ML,1611943156,
505,505,Le Port,-20.9373,55.2919,78.80,73,0,9.22,RE,1611943164,


In [8]:
# Find hotels within 5000 meters
params= {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # Get latitude and longitude from data frame 
    lat = row["Latitude"]
    lng = row["Longitude"]

    # Assign latitude and longitude to location in params
    params["location"] = f"{lat},{lng}"
    
    # Establish base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make API request from URL and return json
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract the results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 58: Najrān.
Missing field/result... skipping.
Retrieving Results for Index 179: Pisco.
Closest hotel is PiscoMar Peru.
Retrieving Results for Index 238: Adrar.
Missing field/result... skipping.
Retrieving Results for Index 254: Tessalit.
Missing field/result... skipping.
Retrieving Results for Index 427: Chato.
Closest hotel is Marine Hotel.
Retrieving Results for Index 439: Abu Dhabi.
Closest hotel is Emirates Palace.
Retrieving Results for Index 452: Araouane.
Missing field/result... skipping.
Retrieving Results for Index 505: Le Port.
Closest hotel is Lodge Roche Tamarin & Spa Village Nature.


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
 # Add marker layer on top of heat map
fig = gmaps.figure()

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

# Export to png
plt.savefig("Heatmap_Hotel")

<Figure size 432x288 with 0 Axes>